In [1]:
from pycmc import *
from neuralimg.crag import crag_utils as cu

/home/morad/third_party/miniconda3/envs/iri/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def read_crag(crag_path, solution=None):
    """ Auxiliar function to read crag, solution and volumes from path"""

    # Read crag
    store = Hdf5CragStore(crag_path)
    crag = Crag()
    store.retrieveCrag(crag)

    # Volumes references
    volume_store = Hdf5VolumeStore(crag_path)

    # Gest proposed solution, if requested
    if solution is None:
        crag_solution = None
    else:
        crag_solution = CragSolution(crag)
        store.retrieveSolution(crag, crag_solution, 'best-effort')

    # Read volumes
    volumes = CragVolumes(crag)

    return crag, volumes, crag_solution

#current_crag = '/DataDisk/morad/cremi/crags/sampleA/mc_heuristic/hdf/training_dataset.h5'
#crag, volumes, solution, ebb, ein = cu.read_crag(current_crag)
current_crag = '/DataDisk/morad/out/project/hdf/training_dataset.h5'
crag, volumes, solution = read_crag(current_crag)

In [3]:
MAX_DEPTH = 5

def _get_connected(crag, node, type_n):
    """ Returns the list of nodes from a type the input node is connected to """
    op = cu.get_opposite(crag, node)
    return [i for i in op if type_n == crag.type(i)]


def get_connected_an(crag, node):
    """ Returns the list of assignment nodes the node is connected to """
    return _get_connected(crag, node, CragNodeType.AssignmentNode)


def get_connected_nan(crag, node):
    """ Returns the list of no assignment nodes the node is connected to """
    return _get_connected(crag, node, CragNodeType.NoAssignmentNode)


def get_edge(crag, node_src, node_dst):
    """ Returns the edge connecting the two nodes """
    adj_edges = [e for e in crag.adjEdges(node_src)]
    for e in adj_edges:
        opposite = crag.oppositeNode(node_src, e)
        if crag.id(opposite) == crag.id(node_dst):
            return e
    return None


def validate_end(crag, sl, solution, volumes, forward=True):
    """ Validates that the assignment ends properly using a no assignment node
    connected to the slice node by a no assignment edge that is part of the solution """
    no_assig = get_connected_nan(crag, sl)
    selected = [solution.selected(i) for i in no_assig]
    # Check one no assignment edge is selected
    assert sum(selected) == 1 or cu.get_depth(sl, volumes) == MAX_DEPTH
    # Assert that each NoAssignmentNode is connected through NoAssignmentEdges
    # And that the edge is part of the solution as well
    for (i, na) in enumerate(no_assig):
        edge = get_edge(crag, sl, na)
        assert crag.type(edge) == CragEdgeType.NoAssignmentEdge
        if selected[i]:
            assert solution.selected(edge) is True

            
def validate_slice(crag, solution, volumes, sl):
    """ Checks that the input slice has only one Assignment Node that
    is part of the best effort for each direction """
    an = get_connected_an(crag, sl) # TODO: has been added to crag_utils
    an_ids = [crag.id(n) for n in an]
    print('Assignment nodes ids {}'.format(an_ids))
    print('Found %d assignment nodes' % len(an))
    sl_depth = cu.get_depth(sl, volumes)
    # Get depths of the other slice nodes that the input node is connected
    # to through the list of assignment nodes
    other_slices = [cu.get_other_slice(crag, sl, i) for i in an]
    other_slices_id = [crag.id(i) for i in other_slices]
    print('Other slices id {}'.format(other_slices_id))
    depths = [cu.get_depth(cu.get_other_slice(crag, sl, i), volumes) for i in an]
    # Get whether they are in the best effort depending on their depth
    backward = [solution.selected(n) for i, n in enumerate(an) if depths[i] < sl_depth]
    forward = [solution.selected(n) for i, n in enumerate(an) if depths[i] > sl_depth]
    print('Backwards: %d, Forward: %d' % (len(backward), len(forward)))
    print('Selected backwards: %d, Selected forward: %d' % (sum(backward), sum(forward)))
    assert sum(backward) <= 1
    assert sum(forward) <= 1
    
    # If any direction does not continue, it must have a non assigmnet node
    if sum(backward) == 0:
        validate_end(crag, sl, solution, volumes, False)
    if sum(forward) == 0:
        validate_end(crag, sl, solution, volumes, True)
    if sum(forward) == 0 and sum(backward) == 0:
        raise Exception('This situation is not possible. Just in case')
    

def validate_assignments(crag, solution, volumes):
    """ Validates that the each Assignment Nodes in the crag that are selected
    have the following properties:
        - Connect with two slice nodes that are part of the solution
        - Connect the slice nodes using AssignmentEdgs that are part of the best effort """
    for n in crag.nodes():
        
        pritn('Found node %d with type' % (crag.id(n), crag.type(n)))
        
        if crag.type(n) == CragNodeType.AssignmentNode:
            if solution.selected(n):

                slices = cu.get_opposite(crag, n)
                
                print('Found assignment node %d connecting %d slices' % (crag.id(n), len(slices)))                
                print('--------------------')
                
                # Check they are slice nodes and that they are part of the solution
                for s in slices:
                    print('Found slice connected to assignment node %d' % (crag.id(n)))
                    assert crag.type(s) == CragNodeType.SliceNode
                    assert solution.selected(s) is True
                    # Check that only one solution exists per slice
                    validate_slice(crag, solution, volumes, s)
                    
                # Check adjacent edges are selected assignment edges
                for e in crag.adjEdges(n):
                    assert crag.type(e) == CragEdgeType.AssignmentEdge
                    assert solution.selected(e) is True
            
def validate(crag, solution, volumes):
    """ TODO """
    print('entry')
    for n in crag.nodes():
        
        print('sdfsadf')
        
        if crag.type(n) == CragNodeType.SliceNode:
            
            edges = crag.adjEdges(n)
            depth = cu.get_depth(n, volumes)

            print('Found slice node of depth %d' % depth)
            
            if depth in [0, 124]:
                continue
            
            print('-------------------')
            print('Number of edges for slice %d is %d' % (crag.id(n), len(edges)))
            print('Slice is solution: ' + str(solution.selected(n)))
            num = sum([solution.selected(e) for e in edges])
            print('Edges connecting slice node: %d ' % num)
            print('Depth of the slice %d' % depth)
            if solution.selected(n) is True: 
                assert num == 2 or (num == 1 and depth in [0, 124])
            else:
                assert num == 0
                

In [ ]:
#validate_assignments(crag, solution, volumes)
validate(crag, solution, volumes)